In [ ]:
import tensorflow as tf
from keras.models import Sequential
import keras.layers
from keras.optimizers import Adam,RMSprop
import numpy as np
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import glob
from skimage import color
from skimage import io
from skimage.filters import threshold_otsu
from keras.models import model_from_yaml
from skimage.transform import resize
import keras
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve

In [ ]:
# Load encoder
yaml_file = open('encoderc.yaml', 'r') #enter path to encoder yaml file
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("encoderc.h5") #enter path to encoder h5 file
print("Loaded model from disk")

In [ ]:
# Compile training set data
X_train_d = np.zeros((800, 32, 32, 6))
Y_train_d = np.zeros((800, 1))
for i in range(1, 401):
    image = f"data_train_annotation/negative_L/img_negative_L_{i}.jpg" #Pass negative training data
    img = resize(io.imread(image), (32,32))
    x1 = loaded_model(img.reshape(1,32,32,3))
    X_train_d[(i-1) * 2, :, :, 0:3] = img
    X_train_d[(i-1) * 2, :, :, 3:6] = x1[0]


    image = f"data_train_annotation/positive_L/img_positive_L_{i}.jpg" #Pass positive training data
    img = resize(io.imread(image), (32,32))
    x1 = loaded_model(img.reshape(1,32,32,3))
    X_train_d[((i-1) * 2) + 1, :, :, 0:3] = img
    X_train_d[((i-1) * 2) + 1, :, :, 3:6] = x1[0]
    Y_train_d[((i-1) * 2) + 1] = 1


In [ ]:
#Compile validation set data
X_vali_d = np.zeros((800, 32, 32, 6))
Y_vali_d = np.zeros((800, 1))
for i in range(1, 401):
    image = f"data_vali_annotation/negative_L/img_negative_L_{i}.jpg" #Pass negative validation data
    img = resize(io.imread(image), (32,32))
    x1 = loaded_model(img.reshape(1,32,32,3))
    X_vali_d[(i-1) * 2, :, :, 0:3] = img
    X_vali_d[(i-1) * 2, :, :, 3:6] = x1[0]


    image = f"data_vali_annotation/positive_L/img_positive_L_{i}.jpg" #Pass positive validation data
    img = resize(io.imread(image), (32,32))
    x1 = loaded_model(img.reshape(1,32,32,3))
    X_vali_d[((i-1) * 2) + 1, :, :, 0:3] = img
    X_vali_d[((i-1) * 2) + 1, :, :, 3:6] = x1[0]
    Y_vali_d[((i-1) * 2) + 1] = 1


In [ ]:
# Compile test set data
X_test_d = np.zeros((800, 32, 32, 6))
Y_test_d = np.zeros((800, 1))
for i in range(1, 401):
    image = f"data_test_annotation/negative_L/img_negative_L_{i}.jpg" #Pass negative testing data
    img = resize(io.imread(image), (32,32))
    x1 = loaded_model(img.reshape(1,32,32,3))
    X_test_d[(i-1) * 2, :, :, 0:3] = img
    X_test_d[(i-1) * 2, :, :, 3:6] = x1[0]


    image = f"data_test_annotation/positive_L/img_positive_L_{i}.jpg" #Pass positive testing data
    img = resize(io.imread(image), (32,32))
    x1 = loaded_model(img.reshape(1,32,32,3))
    X_test_d[((i-1) * 2) + 1, :, :, 0:3] = img
    X_test_d[((i-1) * 2) + 1, :, :, 3:6] = x1[0]
    Y_test_d[((i-1) * 2) + 1] = 1


In [ ]:
# Combine training and validation set data
X_train_f = np.vstack((X_train_d, X_vali_d))
Y_train_f = np.vstack((Y_train_d, Y_vali_d))

In [ ]:
# Classifier network
input_shape = (32, 32, 6)
discriminator = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(3, 3)),
        keras.layers.Conv2D(16, kernel_size=(2, 2), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.8),
        keras.layers.Dense(8, activation="relu"),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
discriminator.summary()


In [ ]:
# Building and training of classifier
discriminator.compile(loss="binary_crossentropy", optimizer=Adam(0.0003, 0.5), metrics=["accuracy"])
discriminator.fit(X_train_f, Y_train_f, epochs = 100, batch_size = 64)

In [ ]:
acc = discriminator.evaluate(X_test_d, Y_test_d)
print(acc)

In [ ]:
index = 21
x = discriminator.predict(X_test_d[index].reshape(1,32,32, 6))
print(x)

METRICS

In [ ]:
discriminator.summary()

In [ ]:
Y_pred_d = np.around(discriminator.predict(X_test_d))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(Y_test_d, Y_pred_d)
print(cm)

In [ ]:
# Precision
pr = precision_score(Y_test_d, Y_pred_d)
print(pr)

In [ ]:
# Recall
pr = recall_score(Y_test_d, Y_pred_d)
print(pr)

In [ ]:
# F1 Score 
f1 = f1_score(Y_test_d, Y_pred_d)
print(f1)

In [ ]:
fpr, tpr, thresholds= roc_curve(Y_test_d, Y_pred_d)

In [ ]:
plt.plot(fpr, tpr)
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')

In [ ]:
# model_yaml = discriminator.to_yaml()
# with open("cnn_model2.yaml", "w") as yaml_file:
#     yaml_file.write(model_yaml)
# # serialize weights to HDF5
# discriminator.save_weights("cnn_model2.h5")
# print("Saved model to disk")